Copy the URL of the YouTube video you want to extract the transcription for below then hit "▶ Run all".

This notebook will then extract the YouTube transcription to a google doc

In [ ]:
url = "https://www.youtube.com/watch?v=nuu-Qk6QJ94"

In [ ]:
google_doc_title = f"youtube generated - VIDEO_ID - VIDEO_TITLE.txt"
video_id = url.split("v=")[1]
!uv pip install youtube-to-docs
from youtube_to_docs.transcript import fetch_transcript
from google.auth import default
from google.colab import auth, userdata
from googleapiclient.discovery import build

auth.authenticate_user()
creds, _ = default()
docs_service = build("docs", "v1", credentials=creds)
drive_service = build("drive", "v3", credentials=creds)

try:
    youtube_service = build(
        "youtube", "v3", developerKey=userdata.get("YOUTUBE_DATA_API_KEY")
    )
    request = youtube_service.videos(
    ).list(part="snippet,contentDetails", id=video_id)
    response = request.execute()
    video_title = response["items"][0]["snippet"]["title"]
except:
    print("Error getting video title")
    video_title = ""
google_doc_title = google_doc_title.replace("VIDEO_ID", video_id)
google_doc_title = google_doc_title.replace("VIDEO_TITLE", video_title)

In [ ]:
content, is_generated, transcript_data = fetch_transcript(video_id)

In [ ]:
doc = (
    docs_service.documents()
    .create(body={"title": google_doc_title})
    .execute()
)
document_id = doc.get("documentId")
requests = [{"insertText": {"location": {"index": 1}, "text": content}}]
docs_service.documents().batchUpdate(
    documentId=document_id, body={"requests": requests}
).execute()
transcript_url = f"https://docs.google.com/document/d/{document_id}"
print(f"Shareable link: {transcript_url}")